<a href="https://colab.research.google.com/github/LucianoTrujillo/datos-tp1/blob/main/7506R_TP1_GRUPO10_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

testing

testing 2
